# **Summary for Colab Implementation**
This project demonstrates an end-to-end pipeline for Intent Detection using BERT in Google Colab. Below is a concise summary of the steps and purpose for each section of the provided Colab implementation:

# 1. **Data Preparation**
Purpose: Convert raw textual queries into a format suitable for BERT input and encode intent labels for classification.
Steps:
Load the dataset (sofmattress_train.csv) containing user queries (sentence) and corresponding intent labels (label).
Tokenize the text using BERT Tokenizer to generate input_ids and attention_masks.
Encode the labels using a numerical mapping (label_map) and split the data into training and validation sets.
Wrap the data in DataLoader objects for efficient batching during training and evaluation.
# 2. **Model Definition and Training**
Purpose: Fine-tune a pre-trained BERT model (bert-base-uncased) for multi-class intent classification.
Steps:
Load BERTForSequenceClassification and customize the number of output labels based on the dataset.
Use weighted CrossEntropyLoss to handle class imbalance.
Train the model for 30 epochs using AdamW optimizer and a learning rate scheduler (StepLR) for gradual decay.
Monitor training progress using average training loss and evaluate on the validation set with metrics like:
Validation Loss
Precision, Recall, F1-Score (via classification_report).
# 3. **Model Saving**
Purpose: Save the trained model, tokenizer, and label map for reuse during inference.
Steps:
Save the fine-tuned model and tokenizer using save_pretrained() into the ./bert_intent_model directory.
Save the reverse label map (label_map.json) to map numeric predictions back to intent names.
# **4. Prediction and Inference**
Purpose: Use the trained model to predict intents for new user queries.
Steps:
Load the saved model, tokenizer, and label map from ./bert_intent_model.
Define a function predict_intent(sentence) to:
Tokenize input sentences.
Perform inference using the model.
Map the numeric prediction back to the corresponding intent name.
Test the model with sample queries (e.g., "Can I return this item?").
# **5. Key Outputs**
Training Loss: Average training loss decreases across epochs, indicating effective model learning.
Validation Metrics:
Accuracy: 91%
Precision, Recall, F1-Score: High performance across most intents with minimal misclassifications.
Sample Predictions:
Sentence: Can I return this item? | Predicted Intent: RETURN_EXCHANGE
Sentence: Why hasn’t my package arrived yet? its been delayed | Predicted Intent: DELAY_IN_DELIVERY
Sentence: Do you offer cash on delivery? | Predicted Intent: COD
Sentence: I'm interested in learning more about your services. | Predicted Intent: LEAD_GEN
Sentence: Could you schedule a call with a support agent for tomorrow? | Predicted Intent: LEAD_GEN


In [6]:
import pandas as pd
from transformers import BertTokenizer
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

file_path = "/content/sofmattress_train.csv"
data = pd.read_csv(file_path)

texts = data['sentence'].tolist()
labels = data['label'].tolist()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_texts(text_list, max_length=128):
    encoded = tokenizer(
        text_list,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    return encoded

max_seq_length = 128
encoded_data = tokenize_texts(texts, max_length=max_seq_length)

input_ids = encoded_data['input_ids']
attention_masks = encoded_data['attention_mask']

label_map = {label: idx for idx, label in enumerate(set(labels))}
labels_encoded = torch.tensor([label_map[label] for label in labels])

train_inputs, val_inputs, train_labels, val_labels = train_test_split(
    input_ids, labels_encoded, test_size=0.2, random_state=42
)
train_masks, val_masks, _, _ = train_test_split(
    attention_masks, labels_encoded, test_size=0.2, random_state=42
)

batch_size = 32

train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
val_dataset = TensorDataset(val_inputs, val_masks, val_labels)

train_dataloader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset),
    batch_size=batch_size
)
val_dataloader = DataLoader(
    val_dataset,
    sampler=SequentialSampler(val_dataset),
    batch_size=batch_size
)

for batch in train_dataloader:
    print("Input IDs shape:", batch[0].shape)
    print("Attention Masks shape:", batch[1].shape)
    print("Labels shape:", batch[2].shape)
    break


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Input IDs shape: torch.Size([32, 128])
Attention Masks shape: torch.Size([32, 128])
Labels shape: torch.Size([32])


In [7]:
BertForSequenceClassification, AdamW
from sklearn.utils.class_weight import compute_class_weight
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
import torch
import numpy as np
from sklearn.metrics import classification_report

# Check for GPU availability
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Load the model
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=len(label_map)
)
model.to(device)

# Calculate class weights to handle imbalance
class_weights = compute_class_weight(
    class_weight="balanced", classes=np.unique(labels_encoded.numpy()), y=labels_encoded.numpy()
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Use weighted loss
loss_fn = CrossEntropyLoss(weight=class_weights)

# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)

# Define epochs and scheduler
epochs =30
total_training_steps = len(train_dataloader) * epochs
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.9)

# Training and Validation Loop
for epoch in range(epochs):
    # Training phase
    model.train()
    total_loss = 0

    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1} Training"):
        batch_input_ids, batch_attention_mask, batch_labels = (
            batch[0].to(device),
            batch[1].to(device),
            batch[2].to(device),
        )

        model.zero_grad()

        outputs = model(
            input_ids=batch_input_ids,
            attention_mask=batch_attention_mask,
        )

        logits = outputs.logits
        loss = loss_fn(logits, batch_labels)

        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}: Average Training Loss = {avg_loss}")

    # Validation phase
    model.eval()
    val_loss = 0
    all_preds = []
    all_labels = []

    for batch in tqdm(val_dataloader, desc=f"Epoch {epoch + 1} Validating"):
        batch_input_ids, batch_attention_mask, batch_labels = (
            batch[0].to(device),
            batch[1].to(device),
            batch[2].to(device),
        )

        with torch.no_grad():
            outputs = model(
                input_ids=batch_input_ids,
                attention_mask=batch_attention_mask,
            )

        logits = outputs.logits
        loss = loss_fn(logits, batch_labels)
        val_loss += loss.item()

        preds = torch.argmax(logits, dim=1).detach().cpu().numpy()
        labels = batch_labels.detach().cpu().numpy()

        all_preds.extend(preds)
        all_labels.extend(labels)

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch + 1}: Average Validation Loss = {avg_val_loss}")

    # Learning rate scheduler step
    scheduler.step()

    # Classification report
    print("Classification Report:")
    print(classification_report(
        all_labels, all_preds, target_names=list(label_map.keys()))
    )

# Save the model and tokenizer
save_directory = "./bert_intent_model"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print("Model and tokenizer saved to ./bert_intent_model")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1 Training: 100%|██████████| 9/9 [05:51<00:00, 39.11s/it]


Epoch 1: Average Training Loss = 3.0831433137257895


Epoch 1 Validating: 100%|██████████| 3/3 [00:26<00:00,  9.00s/it]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

Epoch 1: Average Validation Loss = 3.178724447886149
Classification Report:
                       precision    recall  f1-score   support

   ABOUT_SOF_MATTRESS       0.11      1.00      0.19         3
                  EMI       0.00      0.00      0.00         5
100_NIGHT_TRIAL_OFFER       0.00      0.00      0.00         4
              PILLOWS       0.00      0.00      0.00         3
         DISTRIBUTORS       0.00      0.00      0.00         8
             WARRANTY       0.00      0.00      0.00         5
         ORDER_STATUS       0.00      0.00      0.00         1
     PRODUCT_VARIANTS       0.00      0.00      0.00         2
   SIZE_CUSTOMIZATION       0.00      0.00      0.00         1
         CANCEL_ORDER       0.29      1.00      0.44         2
        CHECK_PINCODE       0.00      0.00      0.00         1
        ERGO_FEATURES       0.00      0.00      0.00         4
             LEAD_GEN       0.00      0.00      0.00         4
                  COD       0.00      0.0

...

Epoch 29: Average Validation Loss = 1.5971521933873494
Classification Report:
                       precision    recall  f1-score   support

   ABOUT_SOF_MATTRESS       1.00      0.67      0.80         3
                  EMI       0.83      1.00      0.91         5
100_NIGHT_TRIAL_OFFER       1.00      0.50      0.67         4
              PILLOWS       1.00      1.00      1.00         3
         DISTRIBUTORS       1.00      0.88      0.93         8
             WARRANTY       1.00      1.00      1.00         5
         ORDER_STATUS       0.33      1.00      0.50         1
     PRODUCT_VARIANTS       0.67      1.00      0.80         2
   SIZE_CUSTOMIZATION       0.50      1.00      0.67         1
         CANCEL_ORDER       1.00      1.00      1.00         2
        CHECK_PINCODE       1.00      1.00      1.00         1
        ERGO_FEATURES       1.00      1.00      1.00         4
             LEAD_GEN       0.50      0.75      0.60         4
                  COD       0.67      1

Epoch 30 Training: 100%|██████████| 9/9 [05:40<00:00, 37.81s/it]


Epoch 30: Average Training Loss = 0.9514422946506076


Epoch 30 Validating: 100%|██████████| 3/3 [00:29<00:00,  9.73s/it]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} 

Epoch 30: Average Validation Loss = 1.5902010599772136
Classification Report:
                       precision    recall  f1-score   support

   ABOUT_SOF_MATTRESS       1.00      0.67      0.80         3
                  EMI       1.00      1.00      1.00         5
100_NIGHT_TRIAL_OFFER       1.00      0.50      0.67         4
              PILLOWS       1.00      1.00      1.00         3
         DISTRIBUTORS       1.00      0.88      0.93         8
             WARRANTY       1.00      1.00      1.00         5
         ORDER_STATUS       0.33      1.00      0.50         1
     PRODUCT_VARIANTS       0.67      1.00      0.80         2
   SIZE_CUSTOMIZATION       0.50      1.00      0.67         1
         CANCEL_ORDER       1.00      1.00      1.00         2
        CHECK_PINCODE       1.00      1.00      1.00         1
        ERGO_FEATURES       1.00      1.00      1.00         4
             LEAD_GEN       0.50      0.75      0.60         4
                  COD       0.50      1

In [13]:
import json
reverse_label_map = {idx: label for label, idx in label_map.items()}
with open(f"{save_directory}/label_map.json", "w") as f:
    json.dump(reverse_label_map, f)
print("Label map saved to ./bert_intent_model/label_map.json")

# Save the model and tokenizer
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print("Model and tokenizer saved to ./bert_intent_model")

Label map saved to ./bert_intent_model/label_map.json
Model and tokenizer saved to ./bert_intent_model


In [9]:
from google.colab import files
import shutil

shutil.make_archive("bert_intent_model", 'zip', "/content/bert_intent_model")
files.download("bert_intent_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
import json
from transformers import BertForSequenceClassification, BertTokenizer
import torch

# Load the model and tokenizer
save_directory = "./bert_intent_model"
model = BertForSequenceClassification.from_pretrained(save_directory)
tokenizer = BertTokenizer.from_pretrained(save_directory)
model.eval()

# Load the reverse label map
with open(f"{save_directory}/label_map.json", "r") as f:
    reverse_label_map = json.load(f)

# Prediction function
def predict_intent(sentence):
    inputs = tokenizer(
        sentence, return_tensors="pt", padding=True, truncation=True, max_length=128
    )
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_label = torch.argmax(logits, dim=1).item()
    return reverse_label_map[str(predicted_label)]

# Test cases
test_cases = [
    "Can I return this item?",
    "Why hasn’t my package arrived yet? its been delayed",
    "Do you offer cash on delivery?",
    "I'm interested in learning more about your services.",
    "Could you schedule a call with a support agent for tomorrow?"
]

for test in test_cases:
    print(f"Sentence: {test} | Predicted Intent: {predict_intent(test)}")


Sentence: Can I return this item? | Predicted Intent: RETURN_EXCHANGE
Sentence: Why hasn’t my package arrived yet? its been delayed | Predicted Intent: DELAY_IN_DELIVERY
Sentence: Do you offer cash on delivery? | Predicted Intent: COD
Sentence: I'm interested in learning more about your services. | Predicted Intent: LEAD_GEN
Sentence: Could you schedule a call with a support agent for tomorrow? | Predicted Intent: LEAD_GEN
